In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch.utils.data as data


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import ast
# Đọc dữ liệu từ file CSV
df = pd.read_csv('data.csv')
    
# Chọn các cột 'x1', 'x2', 'x3', 'x4', 'x5' làm đầu vào và 'y' làm đầu ra
x_data = df[['x1', 'x2', 'x3', 'x4', 'x5']]
y_data = df['y']

# Khởi tạo danh sách để lưu trữ dữ liệu đầu vào dưới dạng tensors
x_train = []

# Lặp qua từng hàng trong x_data
for index in range(len(x_data)):
    # Lấy hàng thứ index từ DataFrame
    row = x_data.iloc[index]
    # Chuyển đổi từng giá trị trong hàng thành list
    x_values = [ast.literal_eval(val) for val in row.values]
    # Thêm tensor tương ứng vào danh sách x_train
    x_train.append(torch.tensor(x_values, dtype=torch.float32))

# Chuyển đổi dữ liệu đầu ra thành tensor PyTorch
y_train = torch.tensor(y_data.values, dtype=torch.long)

# Tạo tensor từ danh sách các tensors đầu vào
X_train = torch.stack(x_train)

# In ra kích thước của tensors đầu vào và đầu ra
print("x_train:", X_train.shape)
print("y_train:", y_train.shape)

x_train: torch.Size([26846, 5, 10])
y_train: torch.Size([26846])


In [4]:
batch_size = 32
length = 10
num_labels = 36
num_sensors = 5

In [5]:
# Tổng số mẫu
total_samples = len(X_train)

# Tính số lượng mẫu cho tập huấn luyện và tập kiểm tra
train_size = int(0.8 * total_samples)  # 80% cho tập huấn luyện
test_size = total_samples - train_size  # 20% cho tập kiểm tra

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_dataset, test_dataset = data.random_split(torch.utils.data.TensorDataset(X_train, y_train),
                                                [train_size, test_size])

# Khởi tạo DataLoader cho cả tập huấn luyện và tập kiểm tra
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### CNN model

In [6]:
# Định nghĩa mô hình CNN 1D
class CNN1D(nn.Module):
    def __init__(self, num_sensors, num_labels, length):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_sensors, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.batch_norm3 = nn.BatchNorm1d(512)
        self.batch_norm4 = nn.BatchNorm1d(512)
        self.batch_norm5 = nn.BatchNorm1d(1024)
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=1)
        self.fc1 = nn.Linear(512 * length, 512)
        self.fc2 = nn.Linear(512, num_labels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.relu(self.batch_norm2(self.conv2(x)))
        x = self.maxpool(x)
        x = self.relu(self.batch_norm3(self.conv3(x)))
        x = self.maxpool(x)
        x = self.relu(self.batch_norm4(self.conv4(x)))
        x = self.maxpool(x)
        x = self.relu(self.batch_norm5(self.conv5(x)))
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [7]:
num_epochs = 100
learning_rate = 0.001

In [9]:
model = CNN1D(num_sensors, num_labels, length)# 7,Create CNN
error = nn.CrossEntropyLoss()#8, Cross Entropy Loss 
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.001)

In [11]:
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        print(labels.shape)
        print(inputs.shape)
        inputs = inputs.view(inputs.size(0), -1)  # Flatten inputs
        print(inputs.shape)
        outputs = model(inputs)
        loss = error(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_loss:.4f}")


torch.Size([32])
torch.Size([32, 5, 10])
torch.Size([32, 50])


RuntimeError: Given groups=1, weight of size [128, 5, 3], expected input[1, 32, 50] to have 5 channels, but got 32 channels instead

In [ ]:

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = error(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")